In [1]:
import pandas as pd
import numpy as np
import scipy

In [2]:
ind_data=pd.read_csv('industry_data/ind_data.csv',header=0)

In [3]:
def org_data(ind_list, index_list, data_frame):
    new_frame=pd.DataFrame()
    for i in range(len(ind_list)):
        for j in range(len(index_list)):
            if j==0:
                new_frame['%s_%s'%(ind_list[i],index_list[j])] = data_frame['%s'%ind_list[i]]
            else:
                new_frame['%s_%s'%(ind_list[i],index_list[j])] = data_frame['%s.%s'%(ind_list[i],j)]
    return new_frame

In [4]:
def modify_data(data_frame, industry_list, index_list):
    for item in index_list:
        for ind in industry_list:
            data_frame['%s_%s'%(ind,item)]=(data_frame['%s_%s'%(ind,item)]/data_frame['%s_C_P'%ind])**(-1)
    return data_frame

In [5]:
def diff(data_frame, diff_index):
    diff=pd.DataFrame(new_frame)
    for item in diff_index:
        diff[str(item)]=(np.diff(data_frame[str(item)])/data_frame[str(item)][:len(data_frame[str(item)])-1])
    return diff

In [6]:
def normalize(data):
    data_new=data/(data[np.argmax(data)]-data[np.argmin(data)])
    return data_new

In [7]:
industry_list=['Energy','Materials','Industrials','Consumer Discretionary','Consumer Staples','Healthcare','Financials','IT','Telecom Services','Utilities']

index_list=['C_P','Vol','PE','PB','PS','PCF']

new_frame=org_data(industry_list, index_list, ind_data)
new_frame=modify_data(new_frame, industry_list, ['PE','PB','PS','PCF'])


HS300=pd.read_excel('HS300.xlsx')


new_frame['HS300']=HS300['Close']

droplist=[]
for i in industry_list:
    droplist.append('%s_C_P'%i)

#new_frame_undrop=diff(new_frame, new_frame.keys()).dropna()
#new_frame_undrop.to_csv('industry_data/undrop_diff_ind.csv',index=False, header=True)

new_frame=new_frame.drop(droplist,axis=1)

new_frame.to_csv('industry_data/org_ind_data.csv', header=True)

In [8]:
import neurolab as nl

In [9]:
diff_data=diff(new_frame, new_frame.keys())
diff_data_ad=pd.DataFrame(diff_data.dropna())
diff_data_ad.to_csv('industry_data/diff_data.csv')

In [10]:
diff_data.dropna().shape

(689, 51)

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

/Users/liyuefan/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [12]:
diff_data_ad['HS300']=map(lambda x: 1 if x>=0 else -1, diff_data_ad['HS300'])

In [13]:
diff_data_ad.shape

(689, 51)

Then normalize data

In [14]:
norm_diff_data=pd.DataFrame()
for key in diff_data_ad.keys():
    if key=='HS300':
        norm_diff_data[str(key)]=diff_data_ad[str(key)]
    else:
        norm_diff_data[str(key)]=normalize(diff_data_ad[str(key)])

In [15]:
norm_diff_data.to_csv('industry_data/data_for_nn.csv',header=True,index=False)

In [16]:
input_nn=norm_diff_data.drop('HS300',axis=1)

In [17]:
input_nn

,Energy_Vol,Energy_PE,Energy_PB,Energy_PS,Energy_PCF,Materials_Vol,Materials_PE,Materials_PB,Materials_PS,Materials_PCF,...,Telecom Services_Vol,Telecom Services_PE,Telecom Services_PB,Telecom Services_PS,Telecom Services_PCF,Utilities_Vol,Utilities_PE,Utilities_PB,Utilities_PS,Utilities_PCF
0,0.319191,-0.001158,0.010535,-0.057060,0.000097,-0.009068,-0.001070,0.108408,-0.010525,0.000992,...,-0.030368,-0.001565,-0.125099,0.126323,-0.000317,0.021887,0.059837,0.037704,0.076076,1.301153e-03
1,-0.128985,-0.008224,0.021049,0.007824,-0.000198,-0.194069,-0.004340,-0.052830,0.029600,-0.001240,...,-0.070138,0.002408,0.094742,-0.033755,0.000090,-0.241776,-0.052157,0.038399,-0.108808,-1.115016e-03
2,-0.154923,0.000625,-0.042012,0.046029,-0.000388,-0.111909,-0.027571,-0.081109,-0.021222,-0.000871,...,-0.060223,0.001102,0.021484,-0.050894,0.000689,-0.087289,0.004902,0.000000,0.000000,1.738168e-04
3,0.304654,0.002498,0.031638,-0.049341,0.000291,0.240303,0.000802,-0.039835,-0.045307,-0.000185,...,0.128897,0.002360,-0.053309,0.148742,0.000191,0.262473,0.335169,-0.090225,0.027119,-4.321280e-04
4,-0.094288,-0.001029,-0.001267,0.038071,-0.000191,-0.158836,-0.002687,0.027986,-0.032453,-0.000735,...,-0.049191,-0.004969,-0.033021,-0.120934,-0.001359,-0.245072,-0.013784,-0.053243,0.010558,-3.483174e-04
5,0.000191,-0.006260,-0.040793,-0.053175,-0.000981,-0.151626,0.010010,0.028248,0.051716,0.000467,...,-0.092459,0.003351,0.076696,0.017600,0.000521,-0.133554,0.011100,0.065237,-0.022340,2.654151e-04
6,0.008487,0.004178,0.052784,0.019619,0.000489,0.252114,-0.010370,-0.083381,-0.021817,0.000141,...,0.235866,0.002826,0.053877,0.070211,0.000937,0.351460,0.065288,-0.034490,-0.049605,-3.531232e-04
7,-0.054994,0.002857,-0.066201,-0.009508,-0.000590,-0.082946,0.017321,0.029045,0.016530,0.000478,...,-0.083778,0.033216,0.045544,-0.104204,0.000298,0.024490,0.017699,-0.003782,0.066909,2.635725e-04
8,-0.068460,-0.004426,0.033515,0.035791,-0.000292,-0.067484,0.002770,0.013843,-0.041446,-0.001341,...,0.013107,-0.000039,-0.175390,0.017647,-0.000619,-0.029798,-0.027893,-0.041078,-0.010408,-1.753083e-04
9,0.184159,0.002094,0.002776,-0.014384,0.000396,0.191240,-0.024965,-0.098220,0.006204,-0.000583,...,0.115048,0.002312,0.108432,0.140797,0.000733,0.142367,0.540256,0.052508,-0.010132,0.000000e+00


In [18]:
input_nn.shape

(689, 50)

In [19]:
train_sample=np.array(input_nn[:500]).reshape(500,50)
test_sample=np.array(input_nn[501:688]).reshape(187,50)

In [20]:
target_nn=np.array(norm_diff_data['HS300'])

In [21]:
train_target=target_nn[1:501].reshape(500,1)
test_target=target_nn[502:689].reshape(187,1)

In [44]:
range_list=[]
for i in range(50):
    range_list.append([-np.max(train_sample[:,i][np.argmax(train_sample[:,i])],np.abs(train_sample[:,i][np.argmin(train_sample[:,i])])),
                      np.max(train_sample[:,i][np.argmax(train_sample[:,i])],np.abs(train_sample[:,i][np.argmin(train_sample[:,i])]))])

In [62]:

# Create network with 50 inputs, 5 neurons in input layer and 1 in output layer
#net = nl.net.newff([[-1, 1], [-1, 1],[-1, 1], [-1, 1],
#                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
#                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
#                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
#                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
#                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
#                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
#                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], 
#                    [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], 
#                    [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1]], 
#                   [10,1])

net=nl.net.newelm(range_list,[3,1])

net.layers[0].initf = nl.init.InitRand([-0.1, 0.1], 'wb')
net.layers[1].initf= nl.init.InitRand([-0.1, 0.1], 'wb')
net.init()

net.train(train_sample,train_target, epochs=1000, goal=0.01, show=15)
# Train process
#err = net.train(train_sample, train_target, show=15)



Epoch: 15; Error: 0.992782859914;
Epoch: 30; Error: 1.01593128252;
Epoch: 45; Error: 0.997586371856;
Epoch: 60; Error: 0.959984776156;
Epoch: 75; Error: 1.02557585843;
Epoch: 90; Error: 0.951841217072;
Epoch: 105; Error: 0.992586503461;
Epoch: 120; Error: 0.941181111455;
Epoch: 135; Error: 0.92807419144;
Epoch: 150; Error: 0.924707013342;
Epoch: 165; Error: 0.917011858007;
Epoch: 180; Error: 0.894588208804;
Epoch: 195; Error: 0.903958698783;
Epoch: 210; Error: 0.877893107232;
Epoch: 225; Error: 0.839438634243;
Epoch: 240; Error: 0.838070188851;
Epoch: 255; Error: 0.855227025524;
Epoch: 270; Error: 0.913002132131;
Epoch: 285; Error: 0.853334797833;
Epoch: 300; Error: 0.848105627465;
Epoch: 315; Error: 0.830479161739;
Epoch: 330; Error: 0.830069674952;
Epoch: 345; Error: 0.84187399232;
Epoch: 360; Error: 0.81988326613;
Epoch: 375; Error: 0.8044568843;
Epoch: 390; Error: 0.797613932819;
Epoch: 405; Error: 0.792133417665;
Epoch: 420; Error: 0.796130843363;
Epoch: 435; Error: 0.821606484782

[0.99931699712869448,
 0.99922676377431685,
 0.99923149350549811,
 0.99913759486185394,
 0.99900069929004076,
 0.9989058801507078,
 0.99868348926636441,
 0.99845448955861316,
 0.9981358535338839,
 0.99768571897828628,
 0.99713600502337385,
 0.99635389011979858,
 0.99540323782005002,
 0.99417570666393107,
 0.99278285991383819,
 0.99122547742007316,
 0.98968196749767912,
 0.98813484177829247,
 0.98638397786471943,
 0.98400701023519521,
 0.98054634957443931,
 0.97601206987016664,
 0.97111141994644334,
 0.96906276988378415,
 1.0156859057940866,
 1.2571981817063846,
 0.96784871697539254,
 1.0535199313756249,
 0.96889525190781556,
 1.0159312825155806,
 1.0318878339693374,
 0.98585598842826128,
 1.0063262259609231,
 1.0364771632012666,
 1.0110783432651944,
 0.99968732731882715,
 1.029762514877534,
 1.0194249218883411,
 1.0025124676549952,
 1.0174164955722509,
 1.0112257574455876,
 1.0019135501102656,
 1.0097187171969346,
 0.99977093656791927,
 0.99758637185633392,
 0.99628279792505392,
 0.984

In [63]:
predict=net.sim(test_sample)

In [64]:
predict

array([[-0.7847943 ],
       [ 0.65470052],
       [ 0.13281217],
       [ 0.85352234],
       [-0.51561766],
       [ 0.80106452],
       [-0.52969649],
       [ 0.7361815 ],
       [ 0.39094682],
       [-0.91955073],
       [ 0.35534014],
       [ 0.81776003],
       [ 0.10883607],
       [-0.95694902],
       [ 0.41803548],
       [-0.04327948],
       [ 0.68147902],
       [ 0.57378658],
       [ 0.99909692],
       [ 0.85282341],
       [ 0.98285892],
       [ 0.22788963],
       [ 0.35181308],
       [ 0.18796739],
       [-0.18204183],
       [ 0.07331701],
       [-0.54228901],
       [ 0.16146712],
       [-0.59887766],
       [-0.10914178],
       [ 0.06584437],
       [ 0.14780676],
       [ 0.40353842],
       [ 0.91056616],
       [ 0.31137506],
       [-0.21155249],
       [-0.07757251],
       [ 0.12423773],
       [ 0.59042053],
       [-0.10344199],
       [-0.18224653],
       [ 0.39427642],
       [ 0.81830031],
       [-0.50424784],
       [-0.60496862],
       [ 0

In [65]:
predict=map(lambda x: 1 if x>=0 else -1, predict)

In [66]:
predict

[-1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1]

In [67]:
count=0
for i in range(len(predict)):
    if predict[i]==test_target[i]:
        count+=1
    else:
        continue

In [68]:
count

90

In [69]:
float(count)/len(predict)

0.48128342245989303

全连接网络的效果不好，四种改进方法：压缩网络节点数（通过原输入-隐含层-输出来观察多少节点可以满足压缩的精度）；bootstrapping（对于时间序列的bootstrapping需要相邻的两两作为一对，进行随机重采样）；重新设计非全连接网络；还有一种忘了2333

先压缩网络

In [74]:
net_2=nl.net.newff(range_list,[10,50])
err_2=net_2.train(train_sample, train_sample, show=30)

Epoch: 30; Error: 809.82087258;
Epoch: 60; Error: 238.038074563;
Epoch: 90; Error: 149.695033947;
Epoch: 120; Error: 91.746008754;
Epoch: 150; Error: 56.202261319;
Epoch: 180; Error: 47.3345249181;
Epoch: 210; Error: 40.4432690449;
Epoch: 240; Error: 35.1754886055;
Epoch: 270; Error: 32.4163508257;
Epoch: 300; Error: 31.244129657;
Epoch: 330; Error: 30.4173168351;
Epoch: 360; Error: 29.762334729;
Epoch: 390; Error: 29.2174134403;
Epoch: 420; Error: 28.7517923888;
Epoch: 450; Error: 28.0087840908;
Epoch: 480; Error: 27.6461410449;
The maximum number of train epochs is reached


In [89]:
for i in range(1,10):
    net_try=nl.net.newff(range_list, [i, 50])
    err_2=net_try.train(train_sample, train_sample, epochs=1000, show=False);
    result_1=net_try.sim(train_sample)
    result_2=net_try.sim(test_sample)
    mse_train=np.sum((result_1-train_sample)**2)/np.sum(train_sample**2)
    mse_test=np.sum((result_2-test_sample)**2)/np.sum(test_sample**2)
    print 'param: %s mse_train:'%i, mse_train, 'mse_test', mse_test

param: 1 mse_train: 127.526315985 mse_test 132.92742439
param: 2 mse_train: 97.9256369632 mse_test 102.790942499
param: 3 mse_train: 11.3696837194 mse_test 12.257199602
param: 4 mse_train: 0.493668878714 mse_test 0.61580732627
param: 5 mse_train: 0.416864202397 mse_test 0.361894035461
param: 6 mse_train: 3.168898188 mse_test 3.27027146001
param: 7 mse_train: 0.359804222151 mse_test 0.311674826616
param: 8 mse_train: 0.298219133868 mse_test 0.321807235315
param: 9 mse_train: 0.264008664777 mse_test 0.2893261426


In [122]:
net_compress=nl.net.newff(range_list, [4, 50])
err_compress=net_compress.train(train_sample, train_sample, epochs=3000, show=50)
compress_mse_test=np.sum((net_compress.sim(test_sample)-test_sample)**2)/np.sum(test_sample**2)

Epoch: 50; Error: 3609.71649531;
Epoch: 100; Error: 2770.64995782;
Epoch: 150; Error: 1376.09583849;
Epoch: 200; Error: 827.312089039;
Epoch: 250; Error: 818.531120963;
Epoch: 300; Error: 813.873760525;
Epoch: 350; Error: 561.456794104;
Epoch: 400; Error: 310.541043753;
Epoch: 450; Error: 144.312813427;
Epoch: 500; Error: 45.5346979214;
Epoch: 550; Error: 42.4196708898;
Epoch: 600; Error: 40.4704716917;
Epoch: 650; Error: 39.416987094;
Epoch: 700; Error: 38.7379872083;
Epoch: 750; Error: 37.5928614132;
Epoch: 800; Error: 36.8778194459;
Epoch: 850; Error: 36.3336604543;
Epoch: 900; Error: 35.7619732447;
Epoch: 950; Error: 35.514209727;
Epoch: 1000; Error: 35.3840965058;
Epoch: 1050; Error: 35.2935584345;
Epoch: 1100; Error: 35.2388328229;
Epoch: 1150; Error: 35.1960553238;
Epoch: 1200; Error: 35.1703013854;
Epoch: 1250; Error: 35.1527584132;
Epoch: 1300; Error: 35.1248322908;
Epoch: 1350; Error: 34.9368553618;
Epoch: 1400; Error: 34.8808031144;
Epoch: 1450; Error: 34.8253780293;
Epoch: 

In [121]:
print compress_mse_test

0.427685740595


In [125]:
net_compress.save

<bound method Net.save of <neurolab.core.Net object at 0x11a3ad690>>

In [129]:
net_compress

In [138]:
from pybrain.structure import *

In [131]:
fnn=FeedForwardNetwork()

In [142]:
inLayer=LinearLayer(50, name='in')
hiddenLayer=TanhLayer(4, name='hidden')
outLayer=LinearLayer(50, name='out')

In [143]:
fnn.addInputModule(inLayer)
fnn.addModule(hiddenLayer)
fnn.addOutputModule(outLayer)

In [144]:
inConnection=FullConnection(inLayer, hiddenLayer)
outConnection=FullConnection(hiddenLayer, outLayer)

In [145]:
fnn.addConnection(inConnection)
fnn.addConnection(outConnection)

In [146]:
fnn.sortModules()

In [149]:
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.datasets.supervised import SupervisedDataSet

In [150]:
DS=SupervisedDataSet(50,50)

In [152]:
train_sample[0]

array([  3.19190934e-01,  -1.15801290e-03,   1.05352237e-02,
        -5.70600981e-02,   9.67168916e-05,  -9.06773665e-03,
        -1.06981529e-03,   1.08408234e-01,  -1.05251695e-02,
         9.91546552e-04,   6.34108876e-03,   2.94167041e-05,
         3.45840616e-02,   1.23325478e-02,   6.26393666e-04,
        -9.98768412e-02,   2.33916693e-02,   7.38279970e-02,
         1.22267798e-02,   1.01884657e-02,   7.29231027e-02,
         2.59043223e-02,   4.39535016e-02,  -3.20266733e-02,
         2.72734019e-04,  -5.56251703e-02,  -4.51942638e-03,
        -2.82654900e-02,  -7.32983556e-03,  -7.29569880e-05,
        -2.90594485e-02,   4.27626538e-03,  -1.31001987e-02,
        -3.52290545e-03,   2.36567745e-04,  -6.59246693e-02,
         8.88957577e-03,   1.57165284e-02,   1.36611156e-02,
         4.72159182e-04,  -3.03684206e-02,  -1.56499573e-03,
        -1.25099123e-01,   1.26323228e-01,  -3.17200972e-04,
         2.18867904e-02,   5.98372442e-02,   3.77039058e-02,
         7.60756372e-02,

In [153]:
for i in range(len(train_sample)):
    DS.addSample(train_sample[i], train_sample[i])

In [156]:
X=DS['input']
y=DS['target']

In [157]:
dataTrain, dataTest=DS.splitWithProportion(0.8)

In [158]:
xTrain=dataTrain['input']
yTrain=dataTrain['target']
xTest=dataTest['input']
yTest=dataTest['target']

In [159]:
trainer=BackpropTrainer(fnn, dataTrain, verbose=True, learningrate=0.01)

In [199]:
trainer.trainUntilConvergence();

Total error: 0.00144218695871
Total error: 0.00144050923272
Total error: 0.00143924148217
Total error: 0.00143819393686
Total error: 0.00143732169778
Total error: 0.00143658233315
Total error: 0.00143591302341
Total error: 0.00143536887275
Total error: 0.00143484268817
Total error: 0.00143436749005
Total error: 0.00143387739295
Total error: 0.00143345116754
Total error: 0.00143302765034
Total error: 0.00143261767645
Total error: 0.001432231359
Total error: 0.00143182662211
Total error: 0.0014314466694
Total error: 0.00143106646234
Total error: 0.00143070971629
Total error: 0.001430318602
train-errors: [  0.001442  0.001441  0.001439  0.001438  0.001437  0.001437  0.001436  0.001435  0.001435  0.001434  0.001434  0.001433  0.001433  0.001433  0.001432  0.001432  0.001431  0.001431  0.001431  0.001430  0.001439]
valid-errors: [  0.001687  0.001675  0.001677  0.001678  0.001680  0.001681  0.001682  0.001683  0.001683  0.001684  0.001685  0.001685  0.001686  0.001686  0.001686  0.001687  0

In [191]:
for mod in fnn.modules:
    if mod.paramdim>0:
        print mod.params

In [201]:
fnn.connections

{<LinearLayer 'in'>: [<FullConnection 'FullConnection-4': 'in' -> 'hidden'>],
 <LinearLayer 'out'>: [],
 <TanhLayer 'hidden'>: [<FullConnection 'FullConnection-3': 'hidden' -> 'out'>]}